# Bicycle and Pedestrian Crashes by Month

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib._color_data import TABLEAU_COLORS
import matplotlib.ticker as mtick

import os
os.chdir('..')

from scripts.crash_data_analysis import CrashDataAnalysis
cda = CrashDataAnalysis()

In [ ]:
# Aggregate fatality crashes by year and day of year

query = """
select
to_char(reported_date, 'YYYY-MM') as crash_year_month
, count(*) as num_crashes
, sum(case when pedestrian_ind > 0 then 1 else 0 end) as pedestrian_involved_crash
, sum(case when bicycle_ind > 0 then 1 else 0 end) as bicycle_involved_crash
, sum(case when pedestrian_ind > 0 and fatality then 1 else 0 end) as pedestrian_involved_fatal_crash
, sum(case when bicycle_ind > 0 and fatality then 1 else 0 end) as bicycle_involved_fatal_crash
, sum(fatality::int) as fatal_crash

from crashes

group by 1 order by 1
"""

crashes_by_month = pd.read_sql(query, cda.conn)

In [ ]:
crashes_by_month['pedestrian_involved_crash_perc'] = (
    crashes_by_month['pedestrian_involved_crash'] / crashes_by_month['num_crashes']
)

crashes_by_month['bicycle_involved_crash_perc'] = (
    crashes_by_month['bicycle_involved_crash'] / crashes_by_month['num_crashes']
)

crashes_by_month['pedestrian_involved_fatal_crash_perc'] = (
    crashes_by_month['pedestrian_involved_fatal_crash'] / crashes_by_month['fatal_crash']
)

crashes_by_month.tail(24)

## Crashes by year

In [ ]:
query = """
select
to_char(reported_date, 'YYYY') as crash_year
, count(*) as num_crashes
, sum(case when pedestrian_ind > 0 then 1 else 0 end) as pedestrian_involved_crash
, sum(case when bicycle_ind > 0 then 1 else 0 end) as bicycle_involved_crash
, sum(case when pedestrian_ind > 0 and fatality then 1 else 0 end) as pedestrian_involved_fatal_crash
, sum(case when bicycle_ind > 0 and fatality then 1 else 0 end) as bicycle_involved_fatal_crash
, sum(fatality::int) as fatal_crash

from crashes

-- where to_char(reported_date, 'YYYY') < '2023'

group by 1 order by 1
"""

crashes_by_year = pd.read_sql(query, cda.conn)

crashes_by_year['pedestrian_involved_crash_perc'] = (
    crashes_by_year['pedestrian_involved_crash'] / crashes_by_year['num_crashes']
)

crashes_by_year['bicycle_involved_crash_perc'] = (
    crashes_by_year['bicycle_involved_crash'] / crashes_by_year['num_crashes']
)

crashes_by_year['pedestrian_involved_fatal_crash_perc'] = (
    crashes_by_year['pedestrian_involved_fatal_crash'] / crashes_by_year['fatal_crash']
)

# todo: calculate crashes per day so the current year can be plotted alongside complete years
# crashes_by_year['pedestrian_involved_crash_per_day'] =

crashes_by_year

## Bicycle crashes by month

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))

data_to_plot = crashes_by_month.copy()

ax1_color = TABLEAU_COLORS['tab:blue']
ax1.bar(
    x=data_to_plot['crash_year_month']
    , height=data_to_plot['bicycle_involved_crash']
    , color=ax1_color
    , alpha=0.5
)
ax1.set_ylabel('Count', color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()

ax2_color = 'black'
ax2.plot(
    data_to_plot['crash_year_month']
    , data_to_plot['bicycle_involved_crash_perc']
    , color=ax2_color
)
ax2.set_ylabel('Percentage of all crashes', color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))

ax1.tick_params(axis='x', rotation=70)

n = 6  # Keep every Nth label
[l.set_visible(False) for (i,l) in enumerate(ax1.xaxis.get_ticklabels()) if i % n != 0]

plt.title('Crashes involving a bicycle in Denver')
plt.savefig('images/crashes_by_month_bicycle.png', facecolor='white', bbox_inches='tight')

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))

data_to_plot = crashes_by_year.copy()

ax1_color = TABLEAU_COLORS['tab:blue']
ax1.bar(
    x=data_to_plot['crash_year']
    , height=data_to_plot['bicycle_involved_crash']
    , color=ax1_color
    , alpha=0.5
)
ax1.set_ylabel('Count', color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()

ax2_color = 'black'
ax2.plot(
    data_to_plot['crash_year']
    , data_to_plot['bicycle_involved_crash_perc']
    , color=ax2_color
)
ax2.set_ylabel('Percentage of all crashes', color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))
ax2.set_ylim(bottom=0, top=0.016)

ax1.tick_params(axis='x', rotation=0)


plt.title('Crashes involving a bicycle in Denver')
plt.savefig('images/crashes_by_year_bicycle.png', facecolor='white', bbox_inches='tight')

## Pedestrian crashes by month

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))

data_to_plot = crashes_by_month.copy()

ax1_color = TABLEAU_COLORS['tab:green']
ax1.bar(
    x=data_to_plot['crash_year_month']
    , height=data_to_plot['pedestrian_involved_crash']
    , color=ax1_color
    , alpha=0.5
)
ax1.set_ylabel('Count', color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()

ax2_color = 'black'
ax2.plot(
    data_to_plot['crash_year_month']
    , data_to_plot['pedestrian_involved_crash_perc']
    , color=ax2_color
)
ax2.set_ylabel('Percentage of all crashes', color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))

ax1.tick_params(axis='x', rotation=70)

n = 6  # Keep every Nth label
[l.set_visible(False) for (i,l) in enumerate(ax1.xaxis.get_ticklabels()) if i % n != 0]

plt.title('Crashes involving a pedestrian in Denver')
plt.savefig('images/crashes_by_month_pedestrian.png', facecolor='white', bbox_inches='tight')

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))

data_to_plot = crashes_by_year.copy()

ax1_color = TABLEAU_COLORS['tab:green']
ax1.bar(
    x=data_to_plot['crash_year']
    , height=data_to_plot['pedestrian_involved_crash']
    , color=ax1_color
    , alpha=0.5
)
ax1.set_ylabel('Count', color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()

ax2_color = 'black'
ax2.plot(
    data_to_plot['crash_year']
    , data_to_plot['pedestrian_involved_crash_perc']
    , color=ax2_color
)
ax2.set_ylabel('Percentage of all crashes', color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))
ax2.set_ylim(bottom=0, top=data_to_plot.pedestrian_involved_crash_perc.max() + 0.005)

ax1.tick_params(axis='x', rotation=0)

plt.title('Crashes involving a pedestrian in Denver')
plt.savefig('images/crashes_by_year_pedestrian.png', facecolor='white', bbox_inches='tight')

## Pedestrian fatal crashes as a percentage of all crashes

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))

data_to_plot = crashes_by_month.copy()

ax1_color = TABLEAU_COLORS['tab:red']
ax1.bar(
    x=data_to_plot['crash_year_month']
    , height=data_to_plot['fatal_crash']
    , color=ax1_color
    , alpha=0.5
)
ax1.set_ylabel('Count', color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()

ax2_color = 'black'
ax2.plot(
    data_to_plot['crash_year_month']
    , data_to_plot['pedestrian_involved_fatal_crash_perc']
    , color=ax2_color
)
ax2.set_ylabel('Percentage of all fatal crashes', color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))

ax1.tick_params(axis='x', rotation=70)

n = 6  # Keep every Nth label
[l.set_visible(False) for (i,l) in enumerate(ax1.xaxis.get_ticklabels()) if i % n != 0]

plt.title('Fatal crashes involving a pedestrian in Denver')
plt.savefig('images/fatal_crashes_by_month_pedestrian.png', facecolor='white', bbox_inches='tight')

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))

data_to_plot = crashes_by_year.copy()

ax1_color = TABLEAU_COLORS['tab:red']
ax1.bar(
    x=data_to_plot['crash_year']
    , height=data_to_plot['fatal_crash']
    , color=ax1_color
    , alpha=0.5
)
ax1.set_ylabel('Count', color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()

ax2_color = 'black'
ax2.plot(
    data_to_plot['crash_year']
    , data_to_plot['pedestrian_involved_fatal_crash_perc']
    , color=ax2_color
)
ax2.set_ylabel('Percentage of all fatal crashes', color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=1))
# ax2.set_ylim(bottom=0, top=0.03)

ax1.tick_params(axis='x', rotation=0)

plt.title('Fatal crashes involving a pedestrian in Denver')
plt.savefig('images/fatal_crashes_by_year_pedestrian.png', facecolor='white', bbox_inches='tight')